# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Preliminary checking

In [91]:
w = data[data.race=='w']
b = data[data.race=='b']

# look at frequencies of interest
print('race counts: black=%d, white=%d'%( 
    (data.race=='b').sum(), (data.race=='w').sum() ))
print('call counts: black=%d, white=%d'%( 
    (b.call==1).sum(), (w.call==1).sum() ))

race counts: black=2435, white=2435
call counts: black=157, white=235


### Helper functions:

In [71]:
def test_null_hypothesis(nulltext, pval, alpha):
    """test null hypothesis and print results"""
    if (pval < alpha):
        print('Result IS sig diff from H( %s )'%(nulltext))
    else:
        print('Result is NOT sig diff from H( %s )'%(nulltext))   
        
def bootstrap_replicate_1d(data, func):
    """datacamp func to draw one bootstrap replicate"""
    return func(np.random.choice(data, size=len(data)))

# datacamp func to draw bootstrap reps
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates
        
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))
    
    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    
    return perm_sample_1, perm_sample_2
    
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### 1. What test is appropriate for this problem? Does CLT apply?
- Based on a little [internet research](https://stattrek.com/hypothesis-test/difference-in-proportions.aspx), it appears that a two-proportion z-test is appropriate. 
- Does CLT apply? Not sure what this means - many of the tests here assume the CLT, so yes?


### 2. What are the null and alternate hypotheses?
- The Null hypothesis is that there is no difference in proportions of calls between resumes with black and white names. 
- The Alternate hypothesis is that there is a difference.

<div class="span5 alert alert-success">
<p>Your answers to Q3 here</p>
</div>

- 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


In [85]:
# bootstrap method:

# Draw bootstrap replicates of the mean call rate
bs_replicates_black = draw_bs_reps(b.call, np.mean,10000)
bs_replicates_white = draw_bs_reps(w.call, np.mean,10000)

# Compute the 95% confidence interval
conf_int_black = np.percentile(bs_replicates_black,[2.5,97.5])
conf_int_white = np.percentile(bs_replicates_white,[2.5,97.5])

moe_black = (max(conf_int_black)-min(conf_int_black))/2
moe_white = (max(conf_int_white)-min(conf_int_white))/2

# Print the confidence interval
print('Bootstrap method:\n')
print('Black named resumes: MOE= +/-%1.4f, CI=[%1.3f,%1.3f]'%
      (moe_black, conf_int_black[0],conf_int_black[1]) )
print('White named resumes: MOE= +/-%1.4f, CI=[%1.3f,%1.3f]'%
      (moe_white, conf_int_white[0],conf_int_white[1]) )

# test null hypothesis that difference between mean 
#  proportions of calls for black vs white resumes is 0

def dif_of_means(sample1,sample2):
    """Difference between means of two samples"""
    dif = np.mean(sample1) - np.mean(sample2)
    return dif

# observed difference in mean calls
dif_obs = np.mean(b.call) - np.mean(w.call)

# Acquire permutation samples
perm_replicates = draw_perm_reps(b.call, w.call, dif_of_means, 10000)

# Compute and print p-value: p
pval = np.sum(perm_replicates <= dif_obs) / len(perm_replicates)

print('\npval = ',pval)

# test the null hypothesis
test_null_hypothesis('mean % black calls = mean % white calls', pval, .05)


Bootstrap method:

Black named resumes: MOE= +/-0.0097, CI=[0.055,0.074]
White named resumes: MOE= +/-0.0119, CI=[0.085,0.108]

pval =  0.0
Result IS sig diff from H( mean % black calls = mean % white calls )


In [92]:
from scipy.stats import norm

# frequentist method:
print('frequentist method:')

# get the necessary count data
count_black = len(b.call)
count_white = len(w.call)
count_black_called = (b.call==1).sum()
count_white_called = (w.call==1).sum()

# proportions of calls for each race type
prop_black_called = count_black_called/count_black
prop_white_called = count_white_called/count_white

# MOE at 95% CI = z(95%) * sd / sqrt(n)
moe_black = 1.96 * np.std(b.call) / np.sqrt(len(b.call))
moe_white = 1.96 * np.std(w.call) / np.sqrt(len(w.call))

# confidence interval, based on mean and MOE
conf_int_black = [np.mean(b.call)-moe_black, np.mean(b.call)+moe_black]
conf_int_white = [np.mean(w.call)-moe_white, np.mean(w.call)+moe_white]

print('mean proportion called: black=%1.3f, white=%1.3f'%(prop_black_called,
                                                         prop_white_called))

print('Black named resumes: MOE= +/-%1.4f, CI=[%1.3f,%1.3f]'%
      (moe_black, conf_int_black[0],conf_int_black[1]) )
print('White named resumes: MOE= +/-%1.4f, CI=[%1.3f,%1.3f]'%
      (moe_white, conf_int_white[0],conf_int_white[1]) )

# pooled sample proportion
pooled_prop = ( (prop_black_called*count_black +
                 prop_white_called*count_white) / 
                 (count_black+count_white) )

# SE of sampling dist difference between two proportions. 
se = np.sqrt(pooled_prop*(1-pooled_prop) / (count_black+count_white))

# z-score
zscore = (prop_black_called-prop_white_called)/se

# get pval based on zscore
pval = norm.cdf(zscore)

print('\nz-score=%1.3f, pval = %1.9f'%(zscore, norm.cdf(zscore)))

test_null_hypothesis('same call rate for black vs white', pval, .05)

frequentist method:
mean proportion called: black=0.064, white=0.097
Black named resumes: MOE= +/-0.0098, CI=[0.055,0.074]
White named resumes: MOE= +/-0.0117, CI=[0.085,0.108]

z-score=-8.217, pval = 0.000000000
Result IS sig diff from H( same call rate for black vs white )


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

4. Write a story describing the statistical significance in the context or the original problem.

This analysis found that otherwise similar resumes with black vs white sounding applicant names get significantly different callback rates. This is strong evidence that racial discrimination influences hiring. A callback is just the first step in getting hired, and other forms of discrimination may continue to add up throughout the hiring process, creating a filtering effect that would explain the near absence of people of color in certain professions. 

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Answer: Given this analysis, it's impossible to know if races is the most important factor. No doubt there are many other factors besides race that strongly affect callback success, including qualifications, education and experience, gender, age, etc. To assess the relative importances I would need to include those factors in the analysis, comparing the strength of each one.